In [ ]:
!pip install torch torchvision torchaudio
!pip install scikit-learn matplotlib seaborn tqdm

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/ISIC

 attribution.txt    ISIC_0000017.jpg   ISIC_0000035.jpg
 ISIC_0000000.jpg   ISIC_0000018.jpg   ISIC_0000036.jpg
 ISIC_0000001.jpg   ISIC_0000019.jpg   ISIC_0000037.jpg
 ISIC_0000002.jpg   ISIC_0000020.jpg   ISIC_0000038.jpg
 ISIC_0000003.jpg   ISIC_0000021.jpg   ISIC_0000039.jpg
 ISIC_0000004.jpg   ISIC_0000022.jpg   ISIC_0000040.jpg
 ISIC_0000005.jpg   ISIC_0000023.jpg   ISIC_0000041.jpg
 ISIC_0000006.jpg   ISIC_0000024.jpg   ISIC_0000042.jpg
 ISIC_0000007.jpg   ISIC_0000025.jpg   ISIC_0000043.jpg
 ISIC_0000008.jpg   ISIC_0000026.jpg   ISIC_0000044.jpg
 ISIC_0000009.jpg   ISIC_0000027.jpg   ISIC_0000045.jpg
 ISIC_0000010.jpg   ISIC_0000028.jpg   ISIC_0000046.jpg
 ISIC_0000011.jpg   ISIC_0000029.jpg   ISIC_0000047.jpg
 ISIC_0000012.jpg   ISIC_0000030.jpg  'ISIC-images (Unzipped Files)'
 ISIC_0000013.jpg   ISIC_0000031.jpg   licenses
 ISIC_0000014.jpg   ISIC_0000032.jpg   metadata.csv
 ISIC_0000015.jpg   ISIC_0000033.jpg   processed
 ISIC_0000016.jpg   ISIC_0000034.jpg


In [ ]:
DATA_DIR = "/content/drive/MyDrive/ISIC/processed"

train_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

train_ds = ImageFolder(DATA_DIR + "/train", transform=train_tf)
test_ds  = ImageFolder(DATA_DIR + "/test", transform=test_tf)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=8, shuffle=False)


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(32*56*56, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

model = SimpleCNN().cuda()


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1, Loss: 2.3364
Epoch 2, Loss: 0.9325
Epoch 3, Loss: 0.6455
Epoch 4, Loss: 0.5658
Epoch 5, Loss: 0.5078
Epoch 6, Loss: 0.4919
Epoch 7, Loss: 0.4439
Epoch 8, Loss: 0.3933
Epoch 9, Loss: 0.3268
Epoch 10, Loss: 0.3224
